In [7]:

import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.core.display import Markdown

import python_tokens


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

genai.configure(api_key=python_tokens.google_api_key)

for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)


model = genai.GenerativeModel('gemini-1.0-pro-latest')

response = model.generate_content("In singapore, There is a taylor swift concert soon, at a stadium of 100000 people. The concert is sold out, how much more transport do we need? Currently, we have 50 bus prepared. Some people might choose taxis, how much aggregate public services do we need to prepare? Do not use math, just aggregate based on previous data, what are some public transports we need to increase? List out some transport services in Singapore as well.")

to_markdown(response.text)



models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


> **Increased Public Transport Needs:**
> 
> * **Buses:** Significantly more than 50 buses will be required to accommodate an estimated 100,000 concert attendees.
> 
> * **MRT (Mass Rapid Transit):** Additional train services and increased frequency on existing lines to handle the surge in commuters.
> 
> * **LRT (Light Rail Transit):** If applicable, additional LRT services to ferry attendees to and from the stadium area.
> 
> * **Special Shuttle Buses:** Dedicated shuttle buses from designated park-and-ride locations to transport attendees to the concert venue.
> 
> **Aggregate Public Services:**
> 
> * **Taxi Services:** Increased number of taxis available for hire, especially after the concert ends.
> 
> * **Ride-Hailing Services (e.g., Grab, Gojek):** Coordination with ride-hailing companies to provide additional vehicles and ensure smoother operations.
> 
> * **Public Bike-Sharing:** Encouraging the use of public bike-sharing programs for shorter distances.
> 
> * **Designated Walking Paths:** Implementing designated walking paths with clear signage to guide attendees to and from the stadium area.
> 
> **Public Transport Services in Singapore:**
> 
> * *Buses (public and private)*
> * *MRT (Mass Rapid Transit)*
> * *LRT (Light Rail Transit)*
> * *Taxis*
> * *Ride-Hailing Services (e.g., Grab, Gojek)*
> * *Public Bike-Sharing Programs*
> * *Private Hire Vehicles (e.g., limousines)*